In [2]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests
import json
import pickle
import csv
import os

In [3]:
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
NID = 3
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

In [26]:
deployer_wallet = KeyWallet.load("./assets/testwallet2.json","qwerty12345!")
print(deployer_wallet.get_address())

hx455dfffb56126d29d3542da8cbaabb865af190a0


In [20]:
#deployer_wallet = KeyWallet.create()
#print(deployer_wallet.get_address()) 

hxb50fd8444ac8d638a9f41db708fac35836061569


In [28]:
DEPLOY_PARAMS = {
    "_initialSupply": 1000,
    "_decimals": 10
}
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('MYT.zip'))\
    .params(DEPLOY_PARAMS)\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

print('txn hash is',tx_hash)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result
    
tx_result = get_tx_result(tx_hash)
print(tx_result)
SCORE_ADDRESS = tx_result['scoreAddress']

txn hash is 0x721a792e594c75bec4655daccaaf2b3fef8cf2a0b3f534d4ef953f16a952c9cb


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x721a792e594c75bec4655daccaaf2b3fef8cf2a0b3f534d4ef953f16a952c9cb', 'blockHeight': 8856868, 'blockHash': '0xe90bfa97a3d5778e0281eb3d57eab8d1caf17f278cb7f596ccaeb5f8fa3e3b82', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cxe0da058f58ef5b42d584ff14a879b851180f5a6d', 'stepUsed': 1059465360, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1059465360, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [31]:
external_methods = ["name", "symbol", "decimals", "totalSupply"]
for method in external_methods:
    call = CallBuilder().from_(deployer_wallet.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method(method)\
                    .build()
    print(icon_service.call(call))

MY TOKEN
MYT
0xa
0x9184e72a000
cxe0da058f58ef5b42d584ff14a879b851180f5a6d


In [39]:
external_methods = [deployer_wallet.get_address(), "hx3ebaf06e2ee517155c879692461872f51b7b1e17"]
for address in external_methods:
    call = CallBuilder().from_(deployer_wallet.get_address())\
                    .to(SCORE_ADDRESS)\
                    .method('balanceOf')\
                    .params({'_owner': address})\
                    .build()
    print(icon_service.call(call))

0x9184e72a000
0x0


In [44]:
params={
    "_to": "hx3ebaf06e2ee517155c879692461872f51b7b1e17",
    "_value": 100,
    "_data": ""
}

call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(SCORE_ADDRESS)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()


estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(call_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

tx_result = get_tx_result(tx_hash)
tx_result

Raised JSONRPCException while returning the custom response. Error message: {'code': -32001, 'message': "'stablecoin' object has no attribute 'Transfer'"}


JSONRPCException: {'code': -32001, 'message': "'stablecoin' object has no attribute 'Transfer'"} (Json rpc error)